In [86]:
import pandas as pd
import config_prod as config
from datetime import date, timedelta
import phood_api
import numpy as np

In [87]:
names = pd.read_excel('Phood Solutions List Names.xlsx')
names=names.drop([0,1,2])
names=names.iloc[:,0:2]

names.columns = ['PLU', 'Item_Names']
names.reset_index(drop=True)
ndic = names.set_index('PLU').to_dict('index')

In [88]:
locations = ['dedham', 'hingham', 'westford', 'westhartford', 'cranston']
unnamedplu = []
masterlist=[]
#print(ndic)
locationinfo = {51: {'name': 'Dedham'}, 52: {'name': 'Hingham'}, 53: {'name': 'Westford'}, 54: {'name': 'West hartford'}, 55: {'name': 'Cranston'}}
for location in locations:
    phoodServer = phood_api.PhoodAPI(config.base_url)
    username = location
    res = phoodServer.login(username, config.password).json()
    endDate = date.today() #- timedelta(days=date.today().weekday())
    startDate = date(2019,10, 1)
    #endDate = date.today()- timedelta(days=1)
    #startDate = endDate - timedelta(days=6)
    res = phoodServer.get_food_logs(startDate, endDate).json()
    PLUEntries = filter(lambda x: x['clientId'], res)
    orderedEntries = list(sorted(PLUEntries, key=lambda x: x["loggedTime"]))
    hotBar,saladBar,soupBar = set(),set(),set()
    res = phoodServer.getPersistentItems().json()
    for pItem in res:
        if pItem['station'].lower().startswith('hot bar'):
            hotBar.add(pItem['clientId'])
        elif pItem['station'].lower().startswith('salad bar'):
            saladBar.add(pItem['clientId'])
        elif pItem['station'].lower().startswith('soup'):
            soupBar.add(pItem['clientId'])
        else:
            print(f'unclassified item {pItem}')

    res = phoodServer.getPanInfo().json()
    panInfo = {}
    for pan in res:
        panInfo[pan["id"]] = {"name": pan["name"], "weight": pan["weightQuantity"]}
    #print(panInfo)
            
    for item in orderedEntries:
        item['clientId'] = item['clientId'].replace("-", "")
        #print(item['clientId'])
        if item['itemName'] in ndic.keys():
            item['itemName'] = ndic[int(item['clientId'])]['Item_Names']
        else:
            unnamedplu.append(item)
        #print(item['Item_Names'])
        if item['clientId'] in hotBar:
            item['station'] = 'HB'
        elif item['clientId'] in saladBar:
            item['station'] = 'SB'
        elif item['clientId'] in soupBar:
            item['station'] = 'SO'
        item['Location'] = locationinfo[item['locationId']]['name']
        item['Served']=item['Saved']=item['Sold']=item['Shrink']=0
        if 'panId' in item and item['panId']:
            panWeight = panInfo[item['panId']]['weight']
        else:
            panWeight = item['panWeight'] 
        if item['actionTaken']=='Served':
            item['Served'] = +item['quantity'] - panWeight
        if item['actionTaken']=='Discarded':
            item['Shrink'] = +item['quantity'] - panWeight
        if item['actionTaken']=='Saved':
            item['Saved'] = +item['quantity'] - panWeight
        item['Sold'] = item['Served'] - item['Saved'] - item['Shrink']
        if item['clientId'] == '00449791900':
            item['station'] = 'HB'
            #print(item)
        
    masterlist.extend(orderedEntries)

In [89]:
#print(startDate)
#print(endDate)

In [90]:
#masterlist

In [91]:
#type(masterlist)

In [92]:
raw = pd.DataFrame(masterlist) 

In [93]:
#raw[raw.clientId == '95501']

In [94]:
raw

,dateProduced,id,itemName,itemType,clientId,loggedTime,meal,actionTaken,actionReason,quantity,...,panWeight,unit,station,employeeName,locationId,Location,Served,Saved,Sold,Shrink
0,2019-09-30,43120,Spring Mix,Menu Item,46000063050,2019-10-01T06:18:52.000Z,Breakfast,Served,Start,6.45954,...,1.94,Lbs,SB,None,51,Dedham,4.51954,0.0,4.51954,0.00000
1,2019-09-30,43122,Romaine Lettuce,Menu Item,46000055863,2019-10-01T06:19:06.000Z,Breakfast,Served,Start,6.45954,...,1.94,Lbs,SB,None,51,Dedham,4.51954,0.0,4.51954,0.00000
2,2019-09-30,43123,Romaine Lettuce,Menu Item,46000055863,2019-10-01T06:19:20.000Z,Breakfast,Served,Start,6.34931,...,1.94,Lbs,SB,None,51,Dedham,4.40931,0.0,4.40931,0.00000
3,2019-09-30,43124,Baby Spinach,Menu Item,46000055923,2019-10-01T06:19:35.000Z,Breakfast,Served,Start,6.76819,...,1.94,Lbs,SB,None,51,Dedham,4.82819,0.0,4.82819,0.00000
4,2019-09-30,43194,Plantains,Menu Item,46000001661,2019-10-01T07:06:17.000Z,Breakfast,Served,Start,6.74615,...,1.94,Lbs,HB,None,51,Dedham,4.80615,0.0,4.80615,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37302,2019-11-19,80868,Mashed Potatoes,Menu Item,46000057474,2019-11-20T21:23:34.000Z,Breakfast,Discarded,Over Produced,8.70826,...,0.95,Lbs,HB,None,55,Cranston,0.00000,0.0,-7.75826,7.75826
37303,2019-11-19,80869,Mashed Potatoes,Menu Item,46000057474,2019-11-20T21:23:51.000Z,Breakfast,Discarded,Over Produced,8.70826,...,0.95,Lbs,HB,None,55,Cranston,0.00000,0.0,-7.75826,7.75826
37304,2019-11-19,80870,Traditional Herb Stuffing,Menu Item,46000020417,2019-11-20T21:24:36.000Z,Breakfast,Discarded,Over Produced,2.53532,...,0.95,Lbs,HB,None,55,Cranston,0.00000,0.0,-1.58532,1.58532
37305,2019-11-19,80871,Waffle Fries,Menu Item,46000064878,2019-11-20T21:26:37.000Z,Breakfast,Discarded,Over Produced,3.24080,...,1.90,Lbs,HB,None,55,Cranston,0.00000,0.0,-1.34080,1.34080


In [95]:
rawfile=raw

In [96]:
raw.columns = ['Date_Produced', 'Log_ID', 'Item_Name', 'Item_Type', 'PLU', 'Logged_Time', 'Meal', 'Action_Taken', 'Action_Reason', 'Quantity', 'Pan_ID', 'Pan_Weight', 'Unit', 'Station', 'Employee_Name', 'Location_ID','Location', 'Served','Saved','Sold','Shrink']


In [97]:
rawfile

,Date_Produced,Log_ID,Item_Name,Item_Type,PLU,Logged_Time,Meal,Action_Taken,Action_Reason,Quantity,...,Pan_Weight,Unit,Station,Employee_Name,Location_ID,Location,Served,Saved,Sold,Shrink
0,2019-09-30,43120,Spring Mix,Menu Item,46000063050,2019-10-01T06:18:52.000Z,Breakfast,Served,Start,6.45954,...,1.94,Lbs,SB,None,51,Dedham,4.51954,0.0,4.51954,0.00000
1,2019-09-30,43122,Romaine Lettuce,Menu Item,46000055863,2019-10-01T06:19:06.000Z,Breakfast,Served,Start,6.45954,...,1.94,Lbs,SB,None,51,Dedham,4.51954,0.0,4.51954,0.00000
2,2019-09-30,43123,Romaine Lettuce,Menu Item,46000055863,2019-10-01T06:19:20.000Z,Breakfast,Served,Start,6.34931,...,1.94,Lbs,SB,None,51,Dedham,4.40931,0.0,4.40931,0.00000
3,2019-09-30,43124,Baby Spinach,Menu Item,46000055923,2019-10-01T06:19:35.000Z,Breakfast,Served,Start,6.76819,...,1.94,Lbs,SB,None,51,Dedham,4.82819,0.0,4.82819,0.00000
4,2019-09-30,43194,Plantains,Menu Item,46000001661,2019-10-01T07:06:17.000Z,Breakfast,Served,Start,6.74615,...,1.94,Lbs,HB,None,51,Dedham,4.80615,0.0,4.80615,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37302,2019-11-19,80868,Mashed Potatoes,Menu Item,46000057474,2019-11-20T21:23:34.000Z,Breakfast,Discarded,Over Produced,8.70826,...,0.95,Lbs,HB,None,55,Cranston,0.00000,0.0,-7.75826,7.75826
37303,2019-11-19,80869,Mashed Potatoes,Menu Item,46000057474,2019-11-20T21:23:51.000Z,Breakfast,Discarded,Over Produced,8.70826,...,0.95,Lbs,HB,None,55,Cranston,0.00000,0.0,-7.75826,7.75826
37304,2019-11-19,80870,Traditional Herb Stuffing,Menu Item,46000020417,2019-11-20T21:24:36.000Z,Breakfast,Discarded,Over Produced,2.53532,...,0.95,Lbs,HB,None,55,Cranston,0.00000,0.0,-1.58532,1.58532
37305,2019-11-19,80871,Waffle Fries,Menu Item,46000064878,2019-11-20T21:26:37.000Z,Breakfast,Discarded,Over Produced,3.24080,...,1.90,Lbs,HB,None,55,Cranston,0.00000,0.0,-1.34080,1.34080


In [98]:
rawfile = rawfile.rename(columns={"Logged_Time": "timestamped"})
rawfile['timestamped']= pd.to_datetime(rawfile['timestamped'], utc = True) 

In [99]:
df = rawfile[rawfile['timestamped']< '2019-11-1']
df

,Date_Produced,Log_ID,Item_Name,Item_Type,PLU,timestamped,Meal,Action_Taken,Action_Reason,Quantity,...,Pan_Weight,Unit,Station,Employee_Name,Location_ID,Location,Served,Saved,Sold,Shrink
0,2019-09-30,43120,Spring Mix,Menu Item,46000063050,2019-10-01 06:18:52+00:00,Breakfast,Served,Start,6.45954,...,1.94,Lbs,SB,None,51,Dedham,4.51954,0.0,4.51954,0.00000
1,2019-09-30,43122,Romaine Lettuce,Menu Item,46000055863,2019-10-01 06:19:06+00:00,Breakfast,Served,Start,6.45954,...,1.94,Lbs,SB,None,51,Dedham,4.51954,0.0,4.51954,0.00000
2,2019-09-30,43123,Romaine Lettuce,Menu Item,46000055863,2019-10-01 06:19:20+00:00,Breakfast,Served,Start,6.34931,...,1.94,Lbs,SB,None,51,Dedham,4.40931,0.0,4.40931,0.00000
3,2019-09-30,43124,Baby Spinach,Menu Item,46000055923,2019-10-01 06:19:35+00:00,Breakfast,Served,Start,6.76819,...,1.94,Lbs,SB,None,51,Dedham,4.82819,0.0,4.82819,0.00000
4,2019-09-30,43194,Plantains,Menu Item,46000001661,2019-10-01 07:06:17+00:00,Breakfast,Served,Start,6.74615,...,1.94,Lbs,HB,None,51,Dedham,4.80615,0.0,4.80615,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35127,2019-10-31,69482,Collard Greens & Tomatoes,Menu Item,46000030030,2019-10-31 21:15:58+00:00,Breakfast,Discarded,Over Produced,5.00449,...,0.95,Lbs,HB,None,55,Cranston,0.00000,0.0,-4.05449,4.05449
35128,2019-10-31,69484,Chicken Tenders,Menu Item,46000007582,2019-10-31 21:16:22+00:00,Breakfast,Discarded,Over Produced,1.76370,...,0.95,Lbs,HB,None,55,Cranston,0.00000,0.0,-0.81370,0.81370
35129,2019-10-31,69485,Waffle Fries,Menu Item,46000064878,2019-10-31 21:16:44+00:00,Breakfast,Discarded,Over Produced,2.55736,...,1.90,Lbs,HB,None,55,Cranston,0.00000,0.0,-0.65736,0.65736
35130,2019-10-31,69486,Potato Samosa,Menu Item,46000057583,2019-10-31 21:17:04+00:00,Breakfast,Discarded,Over Produced,5.46746,...,1.90,Lbs,HB,None,55,Cranston,0.00000,0.0,-3.56746,3.56746


In [100]:
images = pd.read_csv('rek-test-images-all.csv')

In [101]:
images

,timestamp,locationId,imageId,imageKey
0,2019-09-16T17:01:13.585000,51,96fb9610-d8a3-11e9-a35a-0de7ce4e8d91,51/96fb9610d8a311e9a35a0de7ce4e8d91
1,2019-09-16T17:15:20.774000,51,8ff25e60-d8a5-11e9-a35a-0de7ce4e8d91,51/8ff25e60d8a511e9a35a0de7ce4e8d91
2,2019-09-16T17:15:36.551000,51,9959bf70-d8a5-11e9-a35a-0de7ce4e8d91,51/9959bf70d8a511e9a35a0de7ce4e8d91
3,2019-09-16T17:15:52.755000,51,a3024830-d8a5-11e9-ae9b-73759b4dfb23,51/a3024830d8a511e9ae9b73759b4dfb23
4,2019-09-16T17:15:54.492000,51,a40b53c0-d8a5-11e9-ae9b-73759b4dfb23,51/a40b53c0d8a511e9ae9b73759b4dfb23
...,...,...,...,...
33757,2019-11-05T23:56:30.501000,54,e346bd50-0027-11ea-90c3-45c3f76c0e47,54/e346bd50002711ea90c345c3f76c0e47
33758,2019-11-06T00:06:43.342000,54,508edae0-0029-11ea-8246-012e5e367d18,54/508edae0002911ea8246012e5e367d18
33759,2019-11-06T00:20:08.856000,55,30ae8980-002b-11ea-8246-012e5e367d18,55/30ae8980002b11ea8246012e5e367d18
33760,2019-11-06T00:22:27.407000,54,8343bdf0-002b-11ea-90c3-45c3f76c0e47,54/8343bdf0002b11ea90c345c3f76c0e47


In [102]:
images = images.rename(columns={"timestamp": "timestamped"})
images['timestamped']= pd.to_datetime(images['timestamped'], utc = True) 


In [103]:
imagesnew = images[images['timestamped'] >= '2019-10-1']
imagesnew = imagesnew[imagesnew['timestamped'] < '2019-11-1']
imagesnew

,timestamped,locationId,imageId,imageKey
4035,2019-10-01 00:06:49.553000+00:00,52,5d639410-e3df-11e9-8246-012e5e367d18,52/5d639410e3df11e98246012e5e367d18
4036,2019-10-01 00:07:02.519000+00:00,52,651e0870-e3df-11e9-90c3-45c3f76c0e47,52/651e0870e3df11e990c345c3f76c0e47
4037,2019-10-01 00:14:47.436000+00:00,53,7a3abcc0-e3e0-11e9-90c3-45c3f76c0e47,53/7a3abcc0e3e011e990c345c3f76c0e47
4038,2019-10-01 00:18:59.163000+00:00,52,10453ab0-e3e1-11e9-90c3-45c3f76c0e47,52/10453ab0e3e111e990c345c3f76c0e47
4039,2019-10-01 00:20:55.193000+00:00,55,556e0090-e3e1-11e9-8246-012e5e367d18,55/556e0090e3e111e98246012e5e367d18
...,...,...,...,...
31468,2019-10-31 22:35:02.223000+00:00,53,ad91f5f0-fc2e-11e9-8246-012e5e367d18,53/ad91f5f0fc2e11e98246012e5e367d18
31469,2019-10-31 22:35:23.892000+00:00,53,ba7c6340-fc2e-11e9-8246-012e5e367d18,53/ba7c6340fc2e11e98246012e5e367d18
31470,2019-10-31 22:35:41.625000+00:00,53,c50e3a90-fc2e-11e9-90c3-45c3f76c0e47,53/c50e3a90fc2e11e990c345c3f76c0e47
31471,2019-10-31 22:40:20.308000+00:00,55,6b29e140-fc2f-11e9-90c3-45c3f76c0e47,55/6b29e140fc2f11e990c345c3f76c0e47


In [104]:
imagesnew['timestamped'] = pd.to_datetime(imagesnew['timestamped']) - timedelta(hours=4) + timedelta(seconds=20)

In [105]:
imagesnew

,timestamped,locationId,imageId,imageKey
4035,2019-09-30 20:07:09.553000+00:00,52,5d639410-e3df-11e9-8246-012e5e367d18,52/5d639410e3df11e98246012e5e367d18
4036,2019-09-30 20:07:22.519000+00:00,52,651e0870-e3df-11e9-90c3-45c3f76c0e47,52/651e0870e3df11e990c345c3f76c0e47
4037,2019-09-30 20:15:07.436000+00:00,53,7a3abcc0-e3e0-11e9-90c3-45c3f76c0e47,53/7a3abcc0e3e011e990c345c3f76c0e47
4038,2019-09-30 20:19:19.163000+00:00,52,10453ab0-e3e1-11e9-90c3-45c3f76c0e47,52/10453ab0e3e111e990c345c3f76c0e47
4039,2019-09-30 20:21:15.193000+00:00,55,556e0090-e3e1-11e9-8246-012e5e367d18,55/556e0090e3e111e98246012e5e367d18
...,...,...,...,...
31468,2019-10-31 18:35:22.223000+00:00,53,ad91f5f0-fc2e-11e9-8246-012e5e367d18,53/ad91f5f0fc2e11e98246012e5e367d18
31469,2019-10-31 18:35:43.892000+00:00,53,ba7c6340-fc2e-11e9-8246-012e5e367d18,53/ba7c6340fc2e11e98246012e5e367d18
31470,2019-10-31 18:36:01.625000+00:00,53,c50e3a90-fc2e-11e9-90c3-45c3f76c0e47,53/c50e3a90fc2e11e990c345c3f76c0e47
31471,2019-10-31 18:40:40.308000+00:00,55,6b29e140-fc2f-11e9-90c3-45c3f76c0e47,55/6b29e140fc2f11e990c345c3f76c0e47


In [106]:
imagesnew['year'] = imagesnew['timestamped'].dt.year
imagesnew['month'] = imagesnew['timestamped'].dt.month
imagesnew['day'] = imagesnew['timestamped'].dt.day
imagesnew['hour'] = imagesnew['timestamped'].dt.hour
imagesnew['minute'] = imagesnew['timestamped'].dt.minute
imagesnew['second'] = imagesnew['timestamped'].dt.second
imagesnew.loc[(imagesnew.second <= 20) & (imagesnew.second > 0), "second"] = 20
imagesnew.loc[(imagesnew.second <= 40) & (imagesnew.second > 20), "second"] = 40
imagesnew.loc[(imagesnew.second <= 59) & (imagesnew.second > 40), "second"] = 59
#imagesnew['second'] = np.where(imagesnew['second']<=30, 30, 59)
#imagesnew = imagesnew.drop(['second'], 1)

In [107]:
imagesnew

,timestamped,locationId,imageId,imageKey,year,month,day,hour,minute,second
4035,2019-09-30 20:07:09.553000+00:00,52,5d639410-e3df-11e9-8246-012e5e367d18,52/5d639410e3df11e98246012e5e367d18,2019,9,30,20,7,20
4036,2019-09-30 20:07:22.519000+00:00,52,651e0870-e3df-11e9-90c3-45c3f76c0e47,52/651e0870e3df11e990c345c3f76c0e47,2019,9,30,20,7,40
4037,2019-09-30 20:15:07.436000+00:00,53,7a3abcc0-e3e0-11e9-90c3-45c3f76c0e47,53/7a3abcc0e3e011e990c345c3f76c0e47,2019,9,30,20,15,20
4038,2019-09-30 20:19:19.163000+00:00,52,10453ab0-e3e1-11e9-90c3-45c3f76c0e47,52/10453ab0e3e111e990c345c3f76c0e47,2019,9,30,20,19,20
4039,2019-09-30 20:21:15.193000+00:00,55,556e0090-e3e1-11e9-8246-012e5e367d18,55/556e0090e3e111e98246012e5e367d18,2019,9,30,20,21,20
...,...,...,...,...,...,...,...,...,...,...
31468,2019-10-31 18:35:22.223000+00:00,53,ad91f5f0-fc2e-11e9-8246-012e5e367d18,53/ad91f5f0fc2e11e98246012e5e367d18,2019,10,31,18,35,40
31469,2019-10-31 18:35:43.892000+00:00,53,ba7c6340-fc2e-11e9-8246-012e5e367d18,53/ba7c6340fc2e11e98246012e5e367d18,2019,10,31,18,35,59
31470,2019-10-31 18:36:01.625000+00:00,53,c50e3a90-fc2e-11e9-90c3-45c3f76c0e47,53/c50e3a90fc2e11e990c345c3f76c0e47,2019,10,31,18,36,20
31471,2019-10-31 18:40:40.308000+00:00,55,6b29e140-fc2f-11e9-90c3-45c3f76c0e47,55/6b29e140fc2f11e990c345c3f76c0e47,2019,10,31,18,40,40


In [108]:
imagesnew['logtime'] = pd.to_datetime(imagesnew[['year', 'month', 'day', 'hour', 'minute','second']])
imagesnew = imagesnew.drop(['year', 'month', 'day', 'hour', 'minute','second','timestamped'], 1)

In [109]:
imagesnew#.to_csv('join1.csv')
imagesnew

,locationId,imageId,imageKey,logtime
4035,52,5d639410-e3df-11e9-8246-012e5e367d18,52/5d639410e3df11e98246012e5e367d18,2019-09-30 20:07:20
4036,52,651e0870-e3df-11e9-90c3-45c3f76c0e47,52/651e0870e3df11e990c345c3f76c0e47,2019-09-30 20:07:40
4037,53,7a3abcc0-e3e0-11e9-90c3-45c3f76c0e47,53/7a3abcc0e3e011e990c345c3f76c0e47,2019-09-30 20:15:20
4038,52,10453ab0-e3e1-11e9-90c3-45c3f76c0e47,52/10453ab0e3e111e990c345c3f76c0e47,2019-09-30 20:19:20
4039,55,556e0090-e3e1-11e9-8246-012e5e367d18,55/556e0090e3e111e98246012e5e367d18,2019-09-30 20:21:20
...,...,...,...,...
31468,53,ad91f5f0-fc2e-11e9-8246-012e5e367d18,53/ad91f5f0fc2e11e98246012e5e367d18,2019-10-31 18:35:40
31469,53,ba7c6340-fc2e-11e9-8246-012e5e367d18,53/ba7c6340fc2e11e98246012e5e367d18,2019-10-31 18:35:59
31470,53,c50e3a90-fc2e-11e9-90c3-45c3f76c0e47,53/c50e3a90fc2e11e990c345c3f76c0e47,2019-10-31 18:36:20
31471,55,6b29e140-fc2f-11e9-90c3-45c3f76c0e47,55/6b29e140fc2f11e990c345c3f76c0e47,2019-10-31 18:40:40


In [110]:
df['year'] = df['timestamped'].dt.year
df['month'] = df['timestamped'].dt.month
df['day'] = df['timestamped'].dt.day
df['hour'] = df['timestamped'].dt.hour
df['minute'] = df['timestamped'].dt.minute
df['second'] = df['timestamped'].dt.second
df.loc[(df.second <= 20) & (df.second > 0), "second"] = 20
df.loc[(df.second <= 40) & (df.second > 20), "second"] = 40
df.loc[(df.second <= 59) & (df.second > 40), "second"] = 59
#df['second'] = np.where(df['second']<=30, 30, 59)
#df = df.drop(['second'], 1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [111]:
df['logtime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute','second']])
df = df.drop(['year', 'month', 'day', 'hour', 'minute','second','timestamped'], 1)
df = df.sort_values(by='logtime')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [112]:
df#.to_csv('join2.csv')
df

,Date_Produced,Log_ID,Item_Name,Item_Type,PLU,Meal,Action_Taken,Action_Reason,Quantity,Pan_ID,...,Unit,Station,Employee_Name,Location_ID,Location,Served,Saved,Sold,Shrink,logtime
14020,2019-10-01,43100,Organic Rainbow Cherry Tomato,Menu Item,46000016104,Breakfast,Served,Start,3.88014,175,...,Lbs,SB,None,53,Westford,2.03014,0.00000,2.03014,0.0,2019-10-01 05:18:20
14021,2019-10-01,43101,Organic Red Onions,Menu Item,46000055881,Breakfast,Served,Start,2.93215,175,...,Lbs,SB,None,53,Westford,1.08215,0.00000,1.08215,0.0,2019-10-01 05:19:20
14022,2019-10-01,43102,Red Cabbage,Menu Item,46000055765,Breakfast,Served,Start,2.46918,175,...,Lbs,SB,None,53,Westford,0.61918,0.00000,0.61918,0.0,2019-10-01 05:34:59
14023,2019-10-01,43103,Paleo Friendly Roasted Red Bell Peppers,Menu Item,46000028993,Breakfast,Served,Start,2.91010,175,...,Lbs,SB,None,53,Westford,1.06010,0.00000,1.06010,0.0,2019-10-01 05:35:20
14024,2019-10-01,43104,Cooked Red Beets,Menu Item,46000007337,Breakfast,Served,Start,4.05651,175,...,Lbs,SB,None,53,Westford,2.20651,0.00000,2.20651,0.0,2019-10-01 05:36:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4897,2019-10-31,69514,Spring Mix,Menu Item,46000063050,Breakfast,Saved,End,3.74786,110,...,Lbs,SB,None,51,Dedham,0.00000,3.15786,-3.15786,0.0,2019-10-31 21:54:40
4898,2019-10-31,69515,Baby Spinach,Menu Item,46000055923,Breakfast,Saved,End,4.03446,110,...,Lbs,SB,None,51,Dedham,0.00000,3.44446,-3.44446,0.0,2019-10-31 21:54:59
4899,2019-10-31,69516,Arugula,Menu Item,46000055714,Breakfast,Saved,End,1.65347,111,...,Lbs,SB,None,51,Dedham,0.00000,1.30347,-1.30347,0.0,2019-10-31 21:56:40
4900,2019-10-31,69517,Romaine Lettuce,Menu Item,46000055863,Breakfast,Saved,End,5.51156,110,...,Lbs,SB,None,51,Dedham,0.00000,4.92156,-4.92156,0.0,2019-10-31 21:56:59


In [113]:
full = pd.merge(df, imagesnew, on='logtime', how='left')
full

,Date_Produced,Log_ID,Item_Name,Item_Type,PLU,Meal,Action_Taken,Action_Reason,Quantity,Pan_ID,...,Location_ID,Location,Served,Saved,Sold,Shrink,logtime,locationId,imageId,imageKey
0,2019-10-01,43100,Organic Rainbow Cherry Tomato,Menu Item,46000016104,Breakfast,Served,Start,3.88014,175,...,53,Westford,2.03014,0.00000,2.03014,0.0,2019-10-01 05:18:20,NaN,NaN,NaN
1,2019-10-01,43101,Organic Red Onions,Menu Item,46000055881,Breakfast,Served,Start,2.93215,175,...,53,Westford,1.08215,0.00000,1.08215,0.0,2019-10-01 05:19:20,NaN,NaN,NaN
2,2019-10-01,43102,Red Cabbage,Menu Item,46000055765,Breakfast,Served,Start,2.46918,175,...,53,Westford,0.61918,0.00000,0.61918,0.0,2019-10-01 05:34:59,53.0,af870200-e42e-11e9-8246-012e5e367d18,53/af870200e42e11e98246012e5e367d18
3,2019-10-01,43103,Paleo Friendly Roasted Red Bell Peppers,Menu Item,46000028993,Breakfast,Served,Start,2.91010,175,...,53,Westford,1.06010,0.00000,1.06010,0.0,2019-10-01 05:35:20,NaN,NaN,NaN
4,2019-10-01,43104,Cooked Red Beets,Menu Item,46000007337,Breakfast,Served,Start,4.05651,175,...,53,Westford,2.20651,0.00000,2.20651,0.0,2019-10-01 05:36:40,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31560,2019-10-31,69514,Spring Mix,Menu Item,46000063050,Breakfast,Saved,End,3.74786,110,...,51,Dedham,0.00000,3.15786,-3.15786,0.0,2019-10-31 21:54:40,NaN,NaN,NaN
31561,2019-10-31,69515,Baby Spinach,Menu Item,46000055923,Breakfast,Saved,End,4.03446,110,...,51,Dedham,0.00000,3.44446,-3.44446,0.0,2019-10-31 21:54:59,NaN,NaN,NaN
31562,2019-10-31,69516,Arugula,Menu Item,46000055714,Breakfast,Saved,End,1.65347,111,...,51,Dedham,0.00000,1.30347,-1.30347,0.0,2019-10-31 21:56:40,NaN,NaN,NaN
31563,2019-10-31,69517,Romaine Lettuce,Menu Item,46000055863,Breakfast,Saved,End,5.51156,110,...,51,Dedham,0.00000,4.92156,-4.92156,0.0,2019-10-31 21:56:59,NaN,NaN,NaN


In [114]:
full#.to_csv('full.csv')

,Date_Produced,Log_ID,Item_Name,Item_Type,PLU,Meal,Action_Taken,Action_Reason,Quantity,Pan_ID,...,Location_ID,Location,Served,Saved,Sold,Shrink,logtime,locationId,imageId,imageKey
0,2019-10-01,43100,Organic Rainbow Cherry Tomato,Menu Item,46000016104,Breakfast,Served,Start,3.88014,175,...,53,Westford,2.03014,0.00000,2.03014,0.0,2019-10-01 05:18:20,NaN,NaN,NaN
1,2019-10-01,43101,Organic Red Onions,Menu Item,46000055881,Breakfast,Served,Start,2.93215,175,...,53,Westford,1.08215,0.00000,1.08215,0.0,2019-10-01 05:19:20,NaN,NaN,NaN
2,2019-10-01,43102,Red Cabbage,Menu Item,46000055765,Breakfast,Served,Start,2.46918,175,...,53,Westford,0.61918,0.00000,0.61918,0.0,2019-10-01 05:34:59,53.0,af870200-e42e-11e9-8246-012e5e367d18,53/af870200e42e11e98246012e5e367d18
3,2019-10-01,43103,Paleo Friendly Roasted Red Bell Peppers,Menu Item,46000028993,Breakfast,Served,Start,2.91010,175,...,53,Westford,1.06010,0.00000,1.06010,0.0,2019-10-01 05:35:20,NaN,NaN,NaN
4,2019-10-01,43104,Cooked Red Beets,Menu Item,46000007337,Breakfast,Served,Start,4.05651,175,...,53,Westford,2.20651,0.00000,2.20651,0.0,2019-10-01 05:36:40,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31560,2019-10-31,69514,Spring Mix,Menu Item,46000063050,Breakfast,Saved,End,3.74786,110,...,51,Dedham,0.00000,3.15786,-3.15786,0.0,2019-10-31 21:54:40,NaN,NaN,NaN
31561,2019-10-31,69515,Baby Spinach,Menu Item,46000055923,Breakfast,Saved,End,4.03446,110,...,51,Dedham,0.00000,3.44446,-3.44446,0.0,2019-10-31 21:54:59,NaN,NaN,NaN
31562,2019-10-31,69516,Arugula,Menu Item,46000055714,Breakfast,Saved,End,1.65347,111,...,51,Dedham,0.00000,1.30347,-1.30347,0.0,2019-10-31 21:56:40,NaN,NaN,NaN
31563,2019-10-31,69517,Romaine Lettuce,Menu Item,46000055863,Breakfast,Saved,End,5.51156,110,...,51,Dedham,0.00000,4.92156,-4.92156,0.0,2019-10-31 21:56:59,NaN,NaN,NaN


In [115]:
imagesnew.head(5)

,locationId,imageId,imageKey,logtime
4035,52,5d639410-e3df-11e9-8246-012e5e367d18,52/5d639410e3df11e98246012e5e367d18,2019-09-30 20:07:20
4036,52,651e0870-e3df-11e9-90c3-45c3f76c0e47,52/651e0870e3df11e990c345c3f76c0e47,2019-09-30 20:07:40
4037,53,7a3abcc0-e3e0-11e9-90c3-45c3f76c0e47,53/7a3abcc0e3e011e990c345c3f76c0e47,2019-09-30 20:15:20
4038,52,10453ab0-e3e1-11e9-90c3-45c3f76c0e47,52/10453ab0e3e111e990c345c3f76c0e47,2019-09-30 20:19:20
4039,55,556e0090-e3e1-11e9-8246-012e5e367d18,55/556e0090e3e111e98246012e5e367d18,2019-09-30 20:21:20


In [116]:
df51 = df[df.Location_ID==51]
df51#.to_csv('join1.csv')

,Date_Produced,Log_ID,Item_Name,Item_Type,PLU,Meal,Action_Taken,Action_Reason,Quantity,Pan_ID,...,Unit,Station,Employee_Name,Location_ID,Location,Served,Saved,Sold,Shrink,logtime
0,2019-09-30,43120,Spring Mix,Menu Item,46000063050,Breakfast,Served,Start,6.45954,107,...,Lbs,SB,None,51,Dedham,4.51954,0.00000,4.51954,0.0,2019-10-01 06:18:59
2,2019-09-30,43123,Romaine Lettuce,Menu Item,46000055863,Breakfast,Served,Start,6.34931,107,...,Lbs,SB,None,51,Dedham,4.40931,0.00000,4.40931,0.0,2019-10-01 06:19:20
1,2019-09-30,43122,Romaine Lettuce,Menu Item,46000055863,Breakfast,Served,Start,6.45954,107,...,Lbs,SB,None,51,Dedham,4.51954,0.00000,4.51954,0.0,2019-10-01 06:19:20
3,2019-09-30,43124,Baby Spinach,Menu Item,46000055923,Breakfast,Served,Start,6.76819,107,...,Lbs,SB,None,51,Dedham,4.82819,0.00000,4.82819,0.0,2019-10-01 06:19:40
4,2019-09-30,43194,Plantains,Menu Item,46000001661,Breakfast,Served,Start,6.74615,107,...,Lbs,HB,None,51,Dedham,4.80615,0.00000,4.80615,0.0,2019-10-01 07:06:20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4897,2019-10-31,69514,Spring Mix,Menu Item,46000063050,Breakfast,Saved,End,3.74786,110,...,Lbs,SB,None,51,Dedham,0.00000,3.15786,-3.15786,0.0,2019-10-31 21:54:40
4898,2019-10-31,69515,Baby Spinach,Menu Item,46000055923,Breakfast,Saved,End,4.03446,110,...,Lbs,SB,None,51,Dedham,0.00000,3.44446,-3.44446,0.0,2019-10-31 21:54:59
4899,2019-10-31,69516,Arugula,Menu Item,46000055714,Breakfast,Saved,End,1.65347,111,...,Lbs,SB,None,51,Dedham,0.00000,1.30347,-1.30347,0.0,2019-10-31 21:56:40
4900,2019-10-31,69517,Romaine Lettuce,Menu Item,46000055863,Breakfast,Saved,End,5.51156,110,...,Lbs,SB,None,51,Dedham,0.00000,4.92156,-4.92156,0.0,2019-10-31 21:56:59


In [117]:
imagesnew.columns = imagesnew.columns.str.strip()

In [118]:
images51 = imagesnew[imagesnew.locationId==51].reset_index(drop=True)
images51#.to_csv('join2.csv')

,locationId,imageId,imageKey,logtime
0,51,d422df40-e3e2-11e9-8246-012e5e367d18,51/d422df40e3e211e98246012e5e367d18,2019-09-30 20:31:59
1,51,7d0b1370-e3e3-11e9-90c3-45c3f76c0e47,51/7d0b1370e3e311e990c345c3f76c0e47,2019-09-30 20:36:40
2,51,cd0f97a0-e3f3-11e9-90c3-45c3f76c0e47,51/cd0f97a0e3f311e990c345c3f76c0e47,2019-09-30 22:33:40
3,51,d15c24e0-e434-11e9-8246-012e5e367d18,51/d15c24e0e43411e98246012e5e367d18,2019-10-01 06:18:59
4,51,db40f350-e434-11e9-90c3-45c3f76c0e47,51/db40f350e43411e990c345c3f76c0e47,2019-10-01 06:19:20
...,...,...,...,...
5182,51,6b6a6a80-fc20-11e9-90c3-45c3f76c0e47,51/6b6a6a80fc2011e990c345c3f76c0e47,2019-10-31 16:53:20
5183,51,7337f1b0-fc20-11e9-8246-012e5e367d18,51/7337f1b0fc2011e98246012e5e367d18,2019-10-31 16:53:40
5184,51,7d24d670-fc20-11e9-8246-012e5e367d18,51/7d24d670fc2011e98246012e5e367d18,2019-10-31 16:53:59
5185,51,451f7d50-fc22-11e9-8246-012e5e367d18,51/451f7d50fc2211e98246012e5e367d18,2019-10-31 17:06:40


In [119]:
df51 = df51[(df51['logtime'] > '2019-10-2 00:00:00') & (df51['logtime'] < '2019-10-2 23:59:59')]

In [120]:
images51 = images51[(images51['logtime'] > '2019-10-2 00:00:00') & (images51['logtime'] < '2019-10-2 23:59:59')]

In [121]:
images51#.to_csv('join1.csv')

,locationId,imageId,imageKey,logtime
253,51,62903420-e4fc-11e9-90c3-45c3f76c0e47,51/62903420e4fc11e990c345c3f76c0e47,2019-10-02 06:07:40
254,51,8aaff670-e4fc-11e9-90c3-45c3f76c0e47,51/8aaff670e4fc11e990c345c3f76c0e47,2019-10-02 06:08:40
255,51,cf9b8c40-e4fc-11e9-90c3-45c3f76c0e47,51/cf9b8c40e4fc11e990c345c3f76c0e47,2019-10-02 06:10:40
256,51,2ef72b40-e4fd-11e9-90c3-45c3f76c0e47,51/2ef72b40e4fd11e990c345c3f76c0e47,2019-10-02 06:13:20
257,51,64ff9100-e4fd-11e9-90c3-45c3f76c0e47,51/64ff9100e4fd11e990c345c3f76c0e47,2019-10-02 06:14:40
...,...,...,...,...
555,51,cb09ff50-e582-11e9-90c3-45c3f76c0e47,51/cb09ff50e58211e990c345c3f76c0e47,2019-10-02 22:09:40
556,51,d5775eb0-e582-11e9-8246-012e5e367d18,51/d5775eb0e58211e98246012e5e367d18,2019-10-02 22:09:59
557,51,e1110910-e582-11e9-90c3-45c3f76c0e47,51/e1110910e58211e990c345c3f76c0e47,2019-10-02 22:10:20
558,51,eb4cf830-e582-11e9-8246-012e5e367d18,51/eb4cf830e58211e98246012e5e367d18,2019-10-02 22:10:40


In [122]:
df51.to_csv('join2.csv')

In [123]:
#images51['logtime'] = pd.to_datetime(images51['logtime']) - timedelta(hours=4) + timedelta(seconds=20)

In [124]:
images51.to_csv('join1.csv')

In [125]:
images51

,locationId,imageId,imageKey,logtime
253,51,62903420-e4fc-11e9-90c3-45c3f76c0e47,51/62903420e4fc11e990c345c3f76c0e47,2019-10-02 06:07:40
254,51,8aaff670-e4fc-11e9-90c3-45c3f76c0e47,51/8aaff670e4fc11e990c345c3f76c0e47,2019-10-02 06:08:40
255,51,cf9b8c40-e4fc-11e9-90c3-45c3f76c0e47,51/cf9b8c40e4fc11e990c345c3f76c0e47,2019-10-02 06:10:40
256,51,2ef72b40-e4fd-11e9-90c3-45c3f76c0e47,51/2ef72b40e4fd11e990c345c3f76c0e47,2019-10-02 06:13:20
257,51,64ff9100-e4fd-11e9-90c3-45c3f76c0e47,51/64ff9100e4fd11e990c345c3f76c0e47,2019-10-02 06:14:40
...,...,...,...,...
555,51,cb09ff50-e582-11e9-90c3-45c3f76c0e47,51/cb09ff50e58211e990c345c3f76c0e47,2019-10-02 22:09:40
556,51,d5775eb0-e582-11e9-8246-012e5e367d18,51/d5775eb0e58211e98246012e5e367d18,2019-10-02 22:09:59
557,51,e1110910-e582-11e9-90c3-45c3f76c0e47,51/e1110910e58211e990c345c3f76c0e47,2019-10-02 22:10:20
558,51,eb4cf830-e582-11e9-8246-012e5e367d18,51/eb4cf830e58211e98246012e5e367d18,2019-10-02 22:10:40


In [126]:
full = pd.merge(df51, images51, on='logtime', how='left')
full

,Date_Produced,Log_ID,Item_Name,Item_Type,PLU,Meal,Action_Taken,Action_Reason,Quantity,Pan_ID,...,Location_ID,Location,Served,Saved,Sold,Shrink,logtime,locationId,imageId,imageKey
0,2019-10-01,44408,Kale,Menu Item,46000055842,Breakfast,Served,Start,8.35552,106,...,51,Dedham,6.76552,0.00000,6.76552,0.0,2019-10-02 06:07:40,51.0,62903420-e4fc-11e9-90c3-45c3f76c0e47,51/62903420e4fc11e990c345c3f76c0e47
1,2019-10-01,44409,Arugula,Menu Item,46000055714,Breakfast,Served,Start,7.69413,106,...,51,Dedham,6.10413,0.00000,6.10413,0.0,2019-10-02 06:08:40,51.0,8aaff670-e4fc-11e9-90c3-45c3f76c0e47,51/8aaff670e4fc11e990c345c3f76c0e47
2,2019-10-01,44411,Spring Mix,Menu Item,46000063050,Breakfast,Served,Start,6.37136,106,...,51,Dedham,4.78136,0.00000,4.78136,0.0,2019-10-02 06:10:40,51.0,cf9b8c40-e4fc-11e9-90c3-45c3f76c0e47,51/cf9b8c40e4fc11e990c345c3f76c0e47
3,2019-10-01,44416,Romaine Lettuce,Menu Item,46000055863,Breakfast,Served,Start,9.50192,106,...,51,Dedham,7.91192,0.00000,7.91192,0.0,2019-10-02 06:13:20,51.0,2ef72b40-e4fd-11e9-90c3-45c3f76c0e47,51/2ef72b40e4fd11e990c345c3f76c0e47
4,2019-10-01,44420,Romaine Lettuce,Menu Item,46000055863,Breakfast,Served,Start,9.41374,106,...,51,Dedham,7.82374,0.00000,7.82374,0.0,2019-10-02 06:14:40,51.0,64ff9100-e4fd-11e9-90c3-45c3f76c0e47,51/64ff9100e4fd11e990c345c3f76c0e47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,2019-10-02,45641,Summer Squash,Menu Item,46000007477,Breakfast,Saved,End,6.26113,111,...,51,Dedham,0.00000,5.91113,-5.91113,0.0,2019-10-02 22:04:40,NaN,NaN,NaN
269,2019-10-02,45648,Golden Beets,Menu Item,46000055729,Breakfast,Saved,End,9.85466,101,...,51,Dedham,0.00000,7.73466,-7.73466,0.0,2019-10-02 22:06:59,NaN,NaN,NaN
270,2019-10-02,45653,Zucchini,Menu Item,46000007476,Breakfast,Saved,End,8.77440,101,...,51,Dedham,0.00000,6.65440,-6.65440,0.0,2019-10-02 22:10:20,51.0,e1110910-e582-11e9-90c3-45c3f76c0e47,51/e1110910e58211e990c345c3f76c0e47
271,2019-10-02,45655,Snap Peas,Menu Item,46000059426,Breakfast,Saved,End,7.89255,101,...,51,Dedham,0.00000,5.77255,-5.77255,0.0,2019-10-02 22:10:40,51.0,eb4cf830-e582-11e9-8246-012e5e367d18,51/eb4cf830e58211e98246012e5e367d18


In [127]:
full.to_csv('full1.csv')